# Executar Simulações de Deposição

In [17]:
# importação dos objetos
from deposicao import DeposicaoAleatoria, DeposicaoAleatoriaRelaxacaoSuperficial, DeposicaoBalistica
from rng import RNG_PCG
import numpy as np
import pdb
import time
from multiprocessing import Pool

def simular(deposicao):
    deposicao.simular(verbose = True)
    return deposicao

'''
    Descomente a forma de deposição desejada (Aleatoria, Aleatoria com Relaxação
    Superficial ou Balística) e comente as demais.
'''
#forma_deposicao = DeposicaoAleatoria
#forma_deposicao = DeposicaoAleatoriaRelaxacaoSuperficial
#forma_deposicao = DeposicaoBalistica

# numero de nucleos de processamento disponiveis, no nosso caso 4
instancias_paralelas = 4

# o numero de amostras totais, multiplo dos nucleos de processamento
# disponíveis
amostras = 1 * instancias_paralelas

# tempo minimo 10^4
t_min = 10 ** 4

# conseguimos simular subestratos distintos
# recomendamos que simulem de 1 em 1 em cada
# simulação
comprimentos_subestratos = [200, 400, 800, 1600]

if forma_deposicao == DeposicaoAleatoria:
    # na deposição aleatoria, a rugosidade nunca satura, por isso, não
    # faz sentido tempo_maximo > t_min
    z = 0
    coeficiente_tempo_maximo = 0
else:
    coeficiente_tempo_maximo = 1.1
    if forma_deposicao == DeposicaoAleatoriaRelaxacaoSuperficial:
        # na deposicao aleatoria com relaxacao superficial ocorre saturacao
        # da rugosidade quando t > L**2. Neste sentido, , tempo_maximo = 
        # max(10**4, 1.1*L**2)
        z = 2
    elif forma_deposicao == DeposicaoBalistica:
        # na deposicao balistica ocorre saturacao da rugosidade quando t > L**(3/2).
        # Neste sentido, , tempo_maximo = max(10**4, 1.1*L**(3/2))
        z = 3/2

# quantidade de snapshots (ou de instantanenos da simulação da deposição) para
# armazenarmos os dados do nosso interesse: instantaneos do crescimento da in-
# face e rugosidades
janelas = 1000

t_max = lambda L, z = z, c = coeficiente_tempo_maximo, j = janelas:\
    np.int32(np.round(c * (L ** z)) - np.round(c * (L ** z)) % j)

tempo_maximo = {
    # setando tempo_maximo para cada tipo comprimento de subestrato definido em
    # comprimentos_subestratos
    L: max(t_min, t_max(L))
    for L in comprimentos_subestratos
}

dados_medios = {
    L:  {
        'interfaces': np.zeros(L),
        'tempo_maximo': tempo_maximo[L],
        'snapshots': janelas,
        'evolucao_subestratos': np.zeros((janelas ,L)),
        'rugosidades': np.zeros((janelas)),
        'amostras': amostras
    }
    for L in comprimentos_subestratos
}

# instanciando geradores de numeros aleatorios
rngs = [RNG_PCG() for i in range(instancias_paralelas)]

t_inicio = time.time()
for L in comprimentos_subestratos:
    tempo_maximo = dados_medios[L]['tempo_maximo']
    snapshots = dados_medios[L]['snapshots']
    
    # avancar o estado dos geradores de numeros aleatorios para cada instancia
    for i in range(1, instancias_paralelas):
        rngs[i].fast_forward(i * tempo_maximo)
    
    future_steps = 1
    for amostra in range(0,amostras,instancias_paralelas):
        # instancio instancias_paralelas de objetos de deposicao aleatoria
        instancias = []
        for i in range(instancias_paralelas):
            instancias.append(
                forma_deposicao(i, L, rngs[i], tempo_maximo = tempo_maximo, snapshots = snapshots )
            )
        
        begin = time.time()
        # simulacao de instancias_paralelas deposicoes
        print('L: {}, Simulacao das amostras {} a {}'.format(L, amostra + 1, amostra + instancias_paralelas))
        pool = Pool()
        deposicoes = pool.map(simular, instancias)
        pool.close()
        pool.join()
        print('Tempo Gasto: {:.2f}s'.format(time.time() - begin))
        
        # armazenar resultados
        for d in deposicoes:
            dados_medios[L]['evolucao_subestratos'] += d.altura_sitios / amostras
            dados_medios[L]['interfaces'] += d.altura_interface / amostras
            dados_medios[L]['rugosidades'] += d.rugosidade() / amostras
        
        # avancar geradores de numeros aleatorios
        for i in range(instancias_paralelas):
            rngs[i].fast_forward((i + future_steps * instancias_paralelas) * tempo_maximo)
        future_steps += 1
print('Tempo Total Gasto: {:.2f}s'.format(time.time() - t_inicio))

L: 1600, Simulacao das amostras 1 a 4
Instancia 0, 0.0%, 0.00s
Instancia 1, 0.0%, 0.00s
Instancia 2, 0.0%, 0.00s
Instancia 3, 0.0%, 0.00s
Instancia 1, 5.0%, 13.21s
Instancia 2, 5.0%, 13.45s
Instancia 0, 5.0%, 13.59s
Instancia 3, 5.0%, 13.57s
Instancia 1, 10.0%, 26.52s
Instancia 0, 10.0%, 26.92s
Instancia 3, 10.0%, 26.96s
Instancia 2, 10.0%, 27.04s
Instancia 1, 15.0%, 40.99s
Instancia 0, 15.0%, 41.34s
Instancia 2, 15.0%, 41.70s
Instancia 3, 15.0%, 41.95s
Instancia 0, 20.0%, 56.80s
Instancia 1, 20.0%, 56.97s
Instancia 2, 20.0%, 57.31s
Instancia 3, 20.0%, 57.62s
Instancia 0, 25.0%, 72.80s
Instancia 1, 25.0%, 73.22s
Instancia 2, 25.0%, 73.42s
Instancia 3, 25.0%, 73.99s
Instancia 0, 30.0%, 88.61s
Instancia 1, 30.0%, 89.20s
Instancia 2, 30.0%, 89.26s
Instancia 3, 30.0%, 89.59s
Instancia 0, 35.0%, 104.45s
Instancia 2, 35.0%, 105.18s
Instancia 1, 35.0%, 105.25s
Instancia 3, 35.0%, 105.28s
Instancia 0, 40.0%, 120.70s
Instancia 2, 40.0%, 121.14s
Instancia 1, 40.0%, 121.26s
Instancia 3, 40.0%, 12

# Salvar os Dados Para Uso Posterior

In [18]:
import pickle

diretorio = {
    DeposicaoAleatoria: 'DA',
    DeposicaoAleatoriaRelaxacaoSuperficial: 'DARS',
    DeposicaoBalistica: 'DB'
}

for L in comprimentos_subestratos:
    f = open('simulacoes/'+ diretorio[forma_deposicao] +'/L_{}'.format(L), 'wb')
    pickle.dump(dados_medios[L], f)
    f.close()